In [1]:
import numpy as np
import pandas as pd
from random import sample

from coremdlr.config import defaults
from coremdlr.datasets.utils import infer_test_wells

from coremdlr.models import FeaturePredictor, LambdaModel
from coremdlr.datasets import WellLoader, FaciesDataset

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3

In [2]:
import hyperopt
from hyperopt import hp
from hyperopt.pyll.base import scope
from sklearn.metrics import f1_score, log_loss
from sklearn.utils.class_weight import compute_sample_weight

In [3]:
from pathlib import Path

results_dir = Path('/home/ross/Dropbox/core_data/model_results/')

In [4]:
fmodel_args = {
    'logs': {
        'model': 'LambdaModel',
        'model_args': {'feature': 'logs'}
    }
}

XGB_SEARCH_SPACE = {
    'model_type' : 'XGB',
    'max_depth' : scope.int(hp.quniform('max_depth', 3, 10, 1)),
    'learning_rate' : hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators' : scope.int(hp.quniform('n_estimators', 10, 1000, 1)),
    'objective' : 'multi:softprob',
    'n_jobs' : 2,
    'gamma' : hp.uniform('gamma', 0, 0.5),
    'subsample' : hp.uniform('subsample', 0.3, 1),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.3, 1.0),
    'colsample_bylevel' : 1,
    'reg_alpha' : 0,                                    # L1 penalty
    'reg_lambda' : hp.uniform('reg_lambda', 0.1, 10),   # L2 penalty
    'tree_method' : 'gpu_hist',
}

def make_params_usable(params):
    """
    Convert params returned by hyperopt to usable `model_args`
    """
    return {
        **XGB_SEARCH_SPACE,
        **params,
        **{'max_depth' : int(params['max_depth']),
           'n_estimators' : int(params['n_estimators'])}
    }

In [5]:
def run_experiment(num_train, max_evals=10, search_space=XGB_SEARCH_SPACE):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    datasets = [FaciesDataset(s, infer_test_wells(s), features=['logs']) for s in train_sets]
    
    for dset in datasets:
        dset.load_or_generate_data()
    
    def train_model(model_config):
        
        log_losses = []
        
        for dset in datasets:
            
            model = FeaturePredictor(dset, model_args=model_config, feature_model_args=fmodel_args)
            model.fit(dset, verbose=False)
            print('F1 score:', f1_score(dset.y_test, model.predict(dset.X_test), average='macro'))
            
            log_losses.append(log_loss(dset.y_test, model.predict_proba(dset.X_test), 
                                      sample_weight=compute_sample_weight('balanced', dset.y_test)))
            
        return np.mean(log_losses)
    
    return hyperopt.fmin(
        fn=train_model,
        space=XGB_SEARCH_SPACE,
        algo=hyperopt.rand.suggest,
        max_evals=max_evals
    )

In [6]:
best_params_3 = run_experiment(3)

print('\n\n\t Starting TRAIN_SET_6')
best_params_6 = run_experiment(6)

print('\n\n\t Starting TRAIN_SET_9')
best_params_9 = run_experiment(9)

[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
Loading Well:  204-24a-7  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('logs', (19294, 11))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('logs', (13006, 11))]
Loading Well:  204-20-3  from  /home/ross/Dropbox/core_data/facies/train_data


...

Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (3105,)), ('top', (3105,)), ('base', (3105,)), ('logs', (3105, 11))]
Loading Well:  204-20-1  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (11210,)), ('top', (11210,)), ('base', (11210,)), ('logs', (11210, 11))]
Training model for feature:                         
logs                                                
F1 score:                                           
0.2503474740207451                                  
Training model for feature:                         
logs                                                
F1 score:                                           
0.24157010129636353                                 
Training model for feature:                         
logs                                                
F1 score:                                           
0.2734092874303373                                  
Training model for feature:                         
logs                                                
F1 score:                                           
0.24228541738316087                                 
Trainin

F1 score:                                                                    
0.2765836410767407                                                           
Training model for feature:                                                  
logs                                                                         
F1 score:                                                                    
0.25749064520640125                                                          
Training model for feature:                                                  
logs                                                                         
F1 score:                                                                    
0.21661676214389508                                                          
Training model for feature:                                                  
logs                                                                        
F1 score:                                                        

In [7]:
best_params_3

{'colsample_bytree': 0.5895723020171275,
 'gamma': 0.46494597464327164,
 'learning_rate': 0.03567823159227995,
 'max_depth': 8.0,
 'n_estimators': 41.0,
 'reg_lambda': 4.977190069657703,
 'subsample': 0.8169099960548651}

In [8]:
best_params_6

{'colsample_bytree': 0.43985142960938706,
 'gamma': 0.49924188496392,
 'learning_rate': 0.03317205980266823,
 'max_depth': 4.0,
 'n_estimators': 385.0,
 'reg_lambda': 9.423997983617841,
 'subsample': 0.9175205351185038}

In [9]:
best_params_9

{'colsample_bytree': 0.638444401625359,
 'gamma': 0.08821766348448512,
 'learning_rate': 0.03484113066643187,
 'max_depth': 7.0,
 'n_estimators': 126.0,
 'reg_lambda': 4.542000876563912,
 'subsample': 0.494492743900123}

In [10]:
def pred_results_df(num_train, model_config):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    datasets = [FaciesDataset(s, infer_test_wells(s), features=['logs']) for s in train_sets]
    
    pred_dfs = []
    
    for i, dset in enumerate(datasets):
        dset.load_or_generate_data()
        
        model = FeaturePredictor(dset, model_args=model_config, feature_model_args=fmodel_args)
        model.fit(dset, verbose=False)
        
        print('predict_proba: ', model.predict_proba(dset.X_test))
        
        for test_well_name in dset.test_well_names:
            df = model.preds_dataframe(test_well_name, logs=dset.logs_args['which_logs'])
            df['well'] = test_well_name
            df['set_number'] = i
            pred_dfs.append(df)
        
    return pd.concat(pred_dfs)

In [11]:
preds_df_3 = pred_results_df(3, make_params_usable(best_params_3))

(preds_df_3['y_true'] == preds_df_3['y_pred']).sum() / preds_df_3.shape[0]

preds_df_3.to_csv(str(results_dir / 'logs_xgb_results_train_sets3_FINAL.csv'))

[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
Loading Well:  204-24a-7  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('logs', (19294, 11))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('logs', (13006, 11))]
Loading Well:  204-20-3  from  /home/ross/Dropbox/core_data/facies/train_data


...

Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('logs', (19294, 11))]
Loading Well:  204-19-7  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS
Feature shapes:  [('depth', (9807,)), ('top', (9807,)), ('base', (9807,)), ('logs', (9807, 11))]
Loading Well:  205-21b-3  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (3842,)), ('top', (3842,)), ('base', (3842,)), ('logs', (3842, 11))]
Training model for feature:  logs
predict_proba:  [[0.11165812 0.13032134 0.13170171 0.09679046 0.5295284 ]
 [0.11018106 0.12859741 0.12557885 0.09551007 0.54013264]
 [0.11018106 0.12859741 0.12557885 0.09551007 0.54013264]
 ...
 [0.3929786  0.10496154 0.14961895 0.11030419 0.24213673]
 [0.3929786  0.10496154 0.14961895 0.11030419 0.24213673]
 [0.3929786  0.10496154 0.14961895 0.11030419 0.24213673]]


In [12]:
preds_df_6 = pred_results_df(6, make_params_usable(best_params_6))

(preds_df_6['y_true'] == preds_df_6['y_pred']).sum() / preds_df_6.shape[0]

preds_df_6.to_csv(str(results_dir / 'logs_xgb_results_train_sets6_FINAL.csv'))

[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
Loading Well:  204-24a-7  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('logs', (19294, 11))]
Loading Well:  204-24a-6  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (13006,)), ('top', (13006,)), ('base', (13006,)), ('logs', (13006, 11))]
Loading Well:  204-20-3  from  /home/ross/Dropbox/core_data/facies/train_data


...

Feature shapes:  [('depth', (2940,)), ('top', (2940,)), ('base', (2940,)), ('logs', (2940, 11))]
Loading Well:  204-24a-7  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  SP
Adding NaN log:  DTS
Feature shapes:  [('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('logs', (19294, 11))]
Loading Well:  204-19-6  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS
Feature shapes:  [('depth', (1947,)), ('top', (1947,)), ('base', (1947,)), ('logs', (1947, 11))]
Training model for feature:  logs
predict_proba:  [[0.02712866 0.2845192  0.17430416 0.41694498 0.09710297]
 [0.02659508 0.2975369  0.16822468 0.4087442  0.09889913]
 [0.02659508 0.2975369  0.16822468 0.4087442  0.09889913]
 ...
 [0.14627813 0.20102778 0.1020166  0.12428722 0.4263903 ]
 [0.15002225 0.17935106 0.09800103 0.1292409  0.4433848 ]
 [0.14795095 0.20428608 0.09394665 0.12778655 0.42602974]]


In [13]:
preds_df_9 = pred_results_df(9, make_params_usable(best_params_9))

(preds_df_9['y_true'] == preds_df_9['y_pred']).sum() / preds_df_9.shape[0]

preds_df_9.to_csv(str(results_dir / 'logs_xgb_results_train_sets9_FINAL.csv'))

[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
[b'nc', b'bs', b's', b'is', b'ih', b'os', b'sh']
Loading Well:  204-19-3A  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('logs', (9590, 11))]
Loading Well:  204-20-1  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (11210,)), ('top', (11210,)), ('base', (11210,)), ('logs', (11210, 11))]
Loading Well:  204-19-7  from  /home/ross/Dropbox/core_data/facies/train_data


...

Adding NaN log:  DTS
Feature shapes:  [('depth', (9590,)), ('top', (9590,)), ('base', (9590,)), ('logs', (9590, 11))]
Loading Well:  204-20a-7  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (3105,)), ('top', (3105,)), ('base', (3105,)), ('logs', (3105, 11))]
Loading Well:  204-20-1  from  /home/ross/Dropbox/core_data/facies/train_data


Header section Parameter regexp=~P was not found.


Adding NaN log:  DTS1
Adding NaN log:  DTS2
Feature shapes:  [('depth', (11210,)), ('top', (11210,)), ('base', (11210,)), ('logs', (11210, 11))]
Training model for feature:  logs
predict_proba:  [[0.07119007 0.19426715 0.25427255 0.30924687 0.17102331]
 [0.07137315 0.19710961 0.25492647 0.3072274  0.16936338]
 [0.07035443 0.19961166 0.25395417 0.3060556  0.17002413]
 ...
 [0.8265     0.08717781 0.03094419 0.01947612 0.03590193]
 [0.8265     0.08717781 0.03094419 0.01947612 0.03590193]
 [0.8265     0.08717781 0.03094419 0.01947612 0.03590193]]


In [14]:
preds_df_3.shape, preds_df_6.shape, preds_df_9.shape

((279901, 25), (180372, 25), (110801, 25))

In [15]:
#preds_df_3.to_csv('logs_xgb_results_train_sets3.csv')
#preds_df_6.to_csv('logs_xgb_results_train_sets6.csv')
#preds_df_9.to_csv('logs_xgb_results_train_sets9.csv')